# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features[['CHAS','RM','B','DIS','LSTAT']]
y = pd.DataFrame(boston.target, columns = ['target'])

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [5]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

404 102 404 102


Fit the model and apply the model to the make test set predictions

In [6]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Calculate the residuals and the mean squared error

In [7]:
#Calculate the predictions on train and test set
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [8]:
#Calculate the residuals
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

In [9]:
# Calculate the MSE
from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)

In [10]:
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

Train Mean Squarred Error: 20.717854440625164
Test Mean Squarred Error: 25.80053948121028


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [11]:
def kfolds(data, k):
    # Force data as pandas dataframe
    data = pd.DataFrame(data)
    n = len(data)
    num_of_folds = n//k
    remainder = n%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= remainder:
            #Account for remainders by adding 1     # add 1 to fold size to account for leftovers           
            fold = data.iloc[start_obs : start_obs+num_of_folds+1] 
            folds.append(fold)
            start_obs +=  num_of_folds + 1
        else:
            fold = data.iloc[start_obs : start_obs+num_of_folds] 
            folds.append(fold)
            start_obs += num_of_folds
            
    return folds 

### Apply it to the Boston Housing Data

In [12]:
# Make sure to concatenate the data again
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [13]:
bos_folds = kfolds(bos_data, 5)

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [14]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]

    # Fit a linear regression model
    linreg.fit(train[X.columns], train[y.columns])
    
    # Evaluate Train and Test Errors
    y_hat_train = linreg.predict(train[X.columns])
    y_hat_test = linreg.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(mean_squared_error(train[y.columns], y_hat_train))
    test_errs.append(mean_squared_error(test[y.columns], y_hat_test))


# print(train_errs)
print(train_errs)
# print(test_errs)
print(test_errs)

[24.195577370388616, 23.032087348477972, 19.74507285797898, 15.31710138425119, 22.32997280754659]
[13.405144922008224, 17.44401680068203, 37.03271139002068, 58.27954384784228, 26.097988757148396]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring="neg_mean_squared_error")

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [19]:
cv_5_results

array([-13.40514492, -17.4440168 , -37.03271139, -58.27954385,
       -26.09798876])

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!